# 04. MLOps: Data Drift Monitoring

This notebook sets up simple data drift monitoring for our demand forecasting features.

## Setup

In [ ]:
import hopsworks
import pandas as pd
import datetime

# Connect to Hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

## Configuration

In [ ]:
# Configuration
feature_group_name = 'demand_features'
version = 1

## Get Feature Group

In [ ]:
# Get feature group
demand_fg = fs.get_feature_group(name=feature_group_name, version=version)
print(f"✓ Retrieved feature group: {feature_group_name}")

## Set Up Data Drift Monitoring

In [ ]:
# Monitor demand quantity for drift
demand_monitoring = demand_fg.create_feature_monitoring(
    name="demand_drift_monitoring",
    feature_name="repetitive_demand_quantity",
    description="Monitor demand quantity for drift",
    cron_expression="0 0 12 ? * * *"  # Daily at noon
).with_detection_window(
    time_offset="1d"  # Look at yesterday's data
).with_reference_window(
    time_offset="1w1d",  # Compare to same day last week
    window_length="1d"
).compare_on(
    metric="mean",
    threshold=0.2  # Alert if mean changes by >20%
).save()

print("✓ Created demand drift monitoring")

## Check Monitoring Results

In [ ]:
# Get recent monitoring results (after jobs have run)
try:
    results = demand_monitoring.get_history(
        start_time=datetime.datetime.now() - datetime.timedelta(days=7),
        end_time=datetime.datetime.now()
    )
    
    if results:
        print(f"Found {len(results)} monitoring results")
        for r in results:
            print(f"  - {r.execution_time}: mean={r.detection_statistics.mean:.2f}")
    else:
        print("No results yet - monitoring jobs will run on schedule")
except:
    print("No results yet - monitoring jobs will run on schedule")

## Summary

✅ Set up daily monitoring for demand quantity

The monitoring job will:
- Run daily at noon
- Compare yesterday's demand to same day last week
- Alert if mean demand changes by more than 20%

View results in Hopsworks UI under Feature Monitoring.